In [1]:
import numpy as np
import numba

In [2]:
#Creating the sparse matrix.
def sparse_split(filer):
    import csv
    with open(filer, 'r') as file:
        reader = csv.reader(file)
        map_user_to_idx = {}
        map_idx_to_user = []
        map_movie_to_idx = {}
        map_idx_to_movie = []
        data_by_user_train = []
        data_by_user_test = []
        data_by_movie_train = []
        data_by_movie_test = []
        data_by_user = []
        data_by_movie = []
        for row in reader:
            user = row[0]
            movie = row[1]
            rating = row[2]
            
            if user =='userId':
                continue
            if user not in map_user_to_idx:
                map_user_to_idx[user] = len(map_user_to_idx)
                map_idx_to_user.append(user)
                data_by_user_train.append([])
                data_by_user_test.append([])
                data_by_user.append([])
            if movie not in map_movie_to_idx:
                map_movie_to_idx[movie]=len(map_movie_to_idx)
                map_idx_to_movie.append(movie)
                data_by_movie_train.append([])
                data_by_movie_test.append([])
                data_by_movie.append([])
            # flip a coin
            data_by_user[map_user_to_idx[user]].append((map_movie_to_idx[movie],float(rating)))
            data_by_movie[map_movie_to_idx[movie]].append((map_user_to_idx[user],float(rating)))            
            flip = np.random.rand()
            if flip < 0.05:
                data_by_user_test[map_user_to_idx[user]].append((map_movie_to_idx[movie],float(rating)))
                data_by_movie_test[map_movie_to_idx[movie]].append((map_user_to_idx[user],float(rating)))
            else:
                data_by_user_train[map_user_to_idx[user]].append((map_movie_to_idx[movie],float(rating)))
                data_by_movie_train[map_movie_to_idx[movie]].append((map_user_to_idx[user],float(rating)))
    return data_by_user, data_by_movie, data_by_user_train, data_by_movie_train, data_by_user_test, data_by_movie_test, map_idx_to_user, map_idx_to_movie, map_user_to_idx, map_movie_to_idx

In [3]:
#loaing the data to sparse matrix.
path = 'ml-25m/ratings.csv'
data_by_user, data_by_movie, data_by_user_train, data_by_movie_train, data_by_user_test, data_by_movie_test, map_idx_to_user, map_idx_to_movie, map_user_to_idx, map_movie_to_idx = sparse_split(path)

In [4]:
#Preprocess of the dataset for numba.
def user_transform(data_by_user):
    list_for_users = [] #list for number of movies for each user.
    list_for_ratings = [] #list for ratings and movies.
    list_incrementing = [] #list for number of movies incrementing for each user.
    summ = 0
    for i in range(len(data_by_user)):
        list_incrementing.append(summ)
        list_for_users.append(len(data_by_user[i]))
        summ+= len(data_by_user[i])
        for (n,r) in data_by_user[i]:
            rating = list((n,r))
            list_for_ratings.append(rating)
    list_incrementing.append(summ)
    return np.array(list_for_users, dtype = np.int64), np.array(list_for_ratings, dtype = np.float64), np.array(list_incrementing, dtype = np.int64)

In [5]:
#calling the function.
lengths_user, data_array_user, lengths_user_incrementing = user_transform(data_by_user)

In [6]:
#calling the function for the rest of the data.
lengths_movie, data_array_movie, lengths_movie_incrementing = user_transform(data_by_movie)
lengths_user_train, data_array_user_train, lengths_us_train_incrementing = user_transform(data_by_user_train)
lengths_user_test, data_array_user_test, lengths_us_test_incrementing = user_transform(data_by_user_test)
lengths_movie_train, data_array_movie_train, lengths_mv_train_incrementing = user_transform(data_by_movie_train)
lengths_movie_test, data_array_movie_test, lengths_mv_test_incrementing = user_transform(data_by_movie_test)

In [8]:
#function for the updates for the data by user parallellyzed.
from numba import njit, prange
@njit(parallel = True)
def updtating_biases_vector_user(user_biases, lengths_user, lengths_user_incre, movies_biases, user_vector, movie_vector, lamda, gamma, tau, data_by_user):
  for m in prange (len(lengths_user)):
    length1 = lengths_user_incre[m]
    length2 = lengths_user_incre[m+1]
    arr = np.zeros(length2 - length1).astype(np.float64)
    for index_data in prange(length1, length2):
      n = int(data_by_user[index_data, 0])
      index = index_data - length1
      arr[index] = lamda * (data_by_user[index_data,1] - movies_biases[n] - np.dot(user_vector[m,:], movie_vector[n,:]))
    bias = np.sum(arr)
    bias=bias/(lamda*len(arr) + gamma)
    user_biases[m]=bias
  
  for m in prange(len(lengths_user)):
    length1 = lengths_user[m]
    length2 = lengths_user[m+1]
    sum_vn = np.zeros((len(user_vector[m,:]),len(user_vector[m,:])))
    sum_r_vn = np.zeros(len(user_vector[m,:]))
    for indexi_data in range(length1, length2):
      n = int(data_by_user[indexi_data, 0])
      r = data_by_user[indexi_data, 1]
      vn = movie_vector[n,:]
      sum_vn += np.outer(vn,vn)
      sum_r_vn += lamda*(r - user_biases[m] - movies_biases[n])*vn
    sum_vn = lamda*(sum_vn) + tau*np.eye(len(vn))
    user_vector[m,:] = np.linalg.solve(sum_vn, sum_r_vn)
    
  return user_biases, user_vector


  

In [9]:
#function for the updates for the movies parallelyzed.
@njit(parallel = True)
def updtating_biases_vector_movie(user_biases, movies_biases, user_vector, movie_vector, lamda, gamma, tau, lengths_movie, lengths_movie_incre, data_by_movie):
  for n in prange (len(lengths_movie)):
    length1 = lengths_movie_incre[n]
    length2 = lengths_movie_incre[n+1]
    arr = np.zeros(length2 - length1).astype(np.float64)
    for index_data in range(length1, length2):
      m = int(data_by_movie[index_data, 0])
      arr[index_data - length1] = lamda * (data_by_movie[index_data,1] - user_biases[m] - np.dot(user_vector[m,:], movie_vector[n,:]))
    bias = np.sum(arr)
    bias=bias/(lamda*len(arr) + gamma)
    movies_biases[n]=bias

  for n in prange(len(lengths_movie)):
    length1 = lengths_movie_incre[n]
    length2 = lengths_movie_incre[n+1]
    sum_vn = np.zeros((len(movie_vector[n,:]),len(movie_vector[n,:])))
    sum_r_vn = np.zeros(len(movie_vector[n,:]))
    for indexi_data in range(length1, length2):
      m = int(data_by_movie[indexi_data, 0])
      r = data_by_movie[indexi_data, 1]
      vn = user_vector[m,:]
      sum_vn += np.outer(vn,vn)
      sum_r_vn += lamda*(r - user_biases[m] - movies_biases[n])*vn
    sum_vn = lamda*(sum_vn) + tau*np.eye(len(vn))
    movie_vector[n,:] = np.linalg.solve(sum_vn, sum_r_vn)
  return movies_biases, movie_vector

In [10]:
#RMSE calculate parallelized
@njit(parallel = True)
def rmse_calculate(data, lengths, lengths_incre, biases_u, biases_v, vector_u, vector_v):
  arr = np.zeros(len(lengths)).astype(np.float64)
  for n in prange (len(lengths)):
    length1 = lengths_incre[n]
    length2 = lengths_incre[n+1]
    arr1 = np.zeros(length2 - length1).astype(np.float64)
    for indexa_data in prange(length1, length2):
      m = int(data[indexa_data, 0])
      r = data[indexa_data, 1]
      arr1 [indexa_data - length1] = (r-biases_u[n]-biases_v[m]-np.dot(vector_u[n,:], vector_v[m,:]))**2
    arr[n] = np.sum(arr1)
  rmse = np.sum(arr)/lengths_incre[-1]
  return np.sqrt(rmse)

In [11]:
@njit(parallel=True)
def loss_calculate(data, lengths, lengths_incre, biases_u, biases_v, vector_u, vector_v, lamda, gamma, tau):
    arr = np.zeros(len(lengths), dtype=np.float64)
    
    for m in prange(len(lengths)):
        length1 = lengths_incre[m]
        length2 = lengths_incre[m + 1]
        # Slice the relevant portion of the data for user
        data_slice = data[length1:length2]
        loss_sum = 0.0
        
        for indexb_data in range(length2 - length1):
            n = int(data_slice[indexb_data, 0])
            r = data_slice[indexb_data, 1] 
            um = vector_u[m, :]
            vn = vector_v[n, :]
            bm = biases_u[m]
            bn = biases_v[n]
            prediction_error = r - np.dot(um, vn) - bm - bn
            loss = (-lamda / 2) * (prediction_error ** 2)
            regularization = (-gamma / 2) * (bm ** 2 + bn ** 2) - (tau / 2) * (np.dot(um, um) + np.dot(vn, vn))
            loss_sum += loss + regularization
        arr[m] = loss_sum
    # Total loss
    total_loss = np.sum(arr)
    return total_loss


In [20]:
M = len(data_by_user)
N = len(data_by_movie)
user_biases = np.zeros(M)
movies_biases = np.zeros(N)
K=10
p=1/np.sqrt(K)
user_vector = np.random.normal(0,p, (M,K))
movie_vector = np.random.normal(0,p, (N,K))
epoch = 21
lamda = 0.001
gamma = 0.004
tau = 0.002

Loss_train_list = []
Loss_test_list = []
mse_train_list = []
mse_test_list = []

for i in range (epoch):

  user_biases, user_vector  = updtating_biases_vector_user(user_biases, lengths_user, lengths_user_incrementing, movies_biases, user_vector, movie_vector, lamda, gamma, tau, data_array_user)
  movies_biases, movie_vector = updtating_biases_vector_movie(user_biases, movies_biases, user_vector, movie_vector, lamda, gamma, tau, lengths_movie, lengths_movie_incrementing, data_array_movie)
  Loss_train = loss_calculate(data_array_user_train, lengths_user_train, lengths_us_train_incrementing, user_biases, movies_biases, user_vector, movie_vector, lamda, gamma,tau)
  rmse = rmse_calculate(data_array_user_train, lengths_user_train, lengths_us_train_incrementing,  user_biases, movies_biases, user_vector, movie_vector)

  mse_train_list.append(rmse)
  Loss_train_list.append(Loss_train)
  
  Loss_test = loss_calculate(data_array_user_test, lengths_user_test, lengths_us_test_incrementing, user_biases, movies_biases, user_vector, movie_vector, lamda, gamma,tau)
  rmse_test = rmse_calculate(data_array_user_test, lengths_user_test, lengths_us_test_incrementing, user_biases, movies_biases, user_vector, movie_vector)
 
  mse_test_list.append(rmse_test)
  Loss_test_list.append(Loss_test)

  if i<10 or (i>10 and i%10==0):
    print(f'epoch{i+1},  Loss train: {Loss_train}, Loss test: {Loss_test}, rmse : {rmse}, rmse test: {rmse_test}')



epoch1,  Loss train: -604959.9362276391, Loss test: -31847.54811267625, rmse : 0.8655812392398312, rmse test: 0.8655364391261708
epoch2,  Loss train: -586128.3516210453, Loss test: -30854.065659164433, rmse : 0.8546498049280299, rmse test: 0.854594826667045
epoch3,  Loss train: -562890.6186185484, Loss test: -29630.9998433418, rmse : 0.8561907511148253, rmse test: 0.8562281819706602
epoch4,  Loss train: -550606.2450890568, Loss test: -28982.576547057804, rmse : 0.8549080735053737, rmse test: 0.8548590739792917
epoch5,  Loss train: -533251.6449867549, Loss test: -28068.767249079046, rmse : 0.8541953646974269, rmse test: 0.8541319774041506
epoch6,  Loss train: -516656.70475172164, Loss test: -27197.527600578265, rmse : 0.8538555634286119, rmse test: 0.8538328491118862
epoch7,  Loss train: -502269.46077137365, Loss test: -26439.043605573625, rmse : 0.8537280205517792, rmse test: 0.8537306661583373
epoch8,  Loss train: -489253.8008164854, Loss test: -25751.207519847616, rmse : 0.8533114928